# Tratamento e Limpeza de Dados Meteorológicos

Este notebook realiza o tratamento e limpeza dos dados meteorológicos do INMET.


In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

# Configurações de conexão
POSTGRES_HOST = 'postgres'
POSTGRES_USER = 'postgres'
POSTGRES_PASSWORD = 'postgres'
POSTGRES_DB = 'inmet_db'

engine = create_engine(f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:5432/{POSTGRES_DB}')


In [2]:
# Carrega dados do PostgreSQL
query = """
SELECT * FROM dados_meteorologicos
ORDER BY datetime
"""

df = pd.read_sql(query, engine)
print(f"Total de registros: {len(df)}")
print(f"Período: {df['datetime'].min()} a {df['datetime'].max()}")
df.head()


Total de registros: 8784
Período: 2024-01-01 00:00:00 a 2024-12-31 23:00:00


,id,datetime,data,hora_utc,precipitacao,pressao,radiacao,temperatura,umidade,vento_direcao,vento_velocidade,created_at
0,1,2024-01-01 00:00:00,2024-01-01,0000,0.0,1012.4,NaN,23.0,88.0,84.0,4.5,2025-12-01 18:46:41.748766
1,2,2024-01-01 01:00:00,2024-01-01,0100,0.0,1012.8,NaN,23.8,77.0,63.0,2.1,2025-12-01 18:46:41.748766
2,3,2024-01-01 02:00:00,2024-01-01,0200,0.0,1012.6,NaN,23.8,68.0,52.0,1.9,2025-12-01 18:46:41.748766
3,4,2024-01-01 03:00:00,2024-01-01,0300,0.0,1012.3,NaN,24.0,63.0,50.0,1.0,2025-12-01 18:46:41.748766
4,5,2024-01-01 04:00:00,2024-01-01,0400,0.0,1011.8,NaN,23.9,64.0,21.0,1.0,2025-12-01 18:46:41.748766


In [3]:
# Verifica valores nulos
print("Valores nulos por coluna:")
print(df.isnull().sum())
print(f"\nPercentual de nulos:")
print((df.isnull().sum() / len(df) * 100).round(2))


Valores nulos por coluna:
id                     0
datetime               0
data                   0
hora_utc               0
precipitacao        3970
pressao             1488
radiacao            6018
temperatura         3217
umidade             5243
vento_direcao       1489
vento_velocidade    1489
created_at             0
dtype: int64

Percentual de nulos:
id                   0.00
datetime             0.00
data                 0.00
hora_utc             0.00
precipitacao        45.20
pressao             16.94
radiacao            68.51
temperatura         36.62
umidade             59.69
vento_direcao       16.95
vento_velocidade    16.95
created_at           0.00
dtype: float64


In [4]:
# Remove registros completamente vazios
df_clean = df.dropna(subset=['temperatura', 'umidade', 'pressao'], how='all')
print(f"Registros após remoção de vazios: {len(df_clean)}")


Registros após remoção de vazios: 7296


In [5]:
# Interpolação de valores faltantes usando método temporal
df_clean = df_clean.sort_values('datetime')

# Interpolação linear para variáveis contínuas
numeric_cols = ['temperatura', 'umidade', 'pressao', 'radiacao', 
                'vento_velocidade', 'vento_direcao', 'precipitacao']

for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].interpolate(method='linear', limit_direction='both')

print("Valores nulos após interpolação:")
print(df_clean.isnull().sum())


Valores nulos após interpolação:
id                  0
datetime            0
data                0
hora_utc            0
precipitacao        0
pressao             0
radiacao            0
temperatura         0
umidade             0
vento_direcao       0
vento_velocidade    0
created_at          0
dtype: int64


In [6]:
# Remove outliers usando IQR para temperatura (variável alvo)
Q1 = df_clean['temperatura'].quantile(0.25)
Q3 = df_clean['temperatura'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Limites de temperatura: {lower_bound:.2f} a {upper_bound:.2f}°C")

outliers = df_clean[(df_clean['temperatura'] < lower_bound) | (df_clean['temperatura'] > upper_bound)]
print(f"Outliers detectados: {len(outliers)}")

# Remove outliers
df_clean = df_clean[(df_clean['temperatura'] >= lower_bound) & (df_clean['temperatura'] <= upper_bound)]
print(f"Registros após remoção de outliers: {len(df_clean)}")


Limites de temperatura: 16.42 a 31.11°C
Outliers detectados: 135
Registros após remoção de outliers: 7161


In [7]:
# Cria features temporais
df_clean['ano'] = df_clean['datetime'].dt.year
df_clean['mes'] = df_clean['datetime'].dt.month
df_clean['dia'] = df_clean['datetime'].dt.day
df_clean['hora'] = df_clean['datetime'].dt.hour
df_clean['dia_semana'] = df_clean['datetime'].dt.dayofweek
df_clean['dia_ano'] = df_clean['datetime'].dt.dayofyear

# Features cíclicas para capturar padrões sazonais
df_clean['hora_sin'] = np.sin(2 * np.pi * df_clean['hora'] / 24)
df_clean['hora_cos'] = np.cos(2 * np.pi * df_clean['hora'] / 24)
df_clean['mes_sin'] = np.sin(2 * np.pi * df_clean['mes'] / 12)
df_clean['mes_cos'] = np.cos(2 * np.pi * df_clean['mes'] / 12)

df_clean.head()


,id,datetime,data,hora_utc,precipitacao,pressao,radiacao,temperatura,umidade,vento_direcao,...,ano,mes,dia,hora,dia_semana,dia_ano,hora_sin,hora_cos,mes_sin,mes_cos
0,1,2024-01-01 00:00:00,2024-01-01,0000,0.0,1012.4,88.3,23.0,88.0,84.0,...,2024,1,1,0,0,1,0.000000,1.000000,0.5,0.866025
1,2,2024-01-01 01:00:00,2024-01-01,0100,0.0,1012.8,88.3,23.8,77.0,63.0,...,2024,1,1,1,0,1,0.258819,0.965926,0.5,0.866025
2,3,2024-01-01 02:00:00,2024-01-01,0200,0.0,1012.6,88.3,23.8,68.0,52.0,...,2024,1,1,2,0,1,0.500000,0.866025,0.5,0.866025
3,4,2024-01-01 03:00:00,2024-01-01,0300,0.0,1012.3,88.3,24.0,63.0,50.0,...,2024,1,1,3,0,1,0.707107,0.707107,0.5,0.866025
4,5,2024-01-01 04:00:00,2024-01-01,0400,0.0,1011.8,88.3,23.9,64.0,21.0,...,2024,1,1,4,0,1,0.866025,0.500000,0.5,0.866025


In [8]:
# Salva dados tratados em nova tabela
df_clean.to_sql('dados_tratados', engine, if_exists='replace', index=False)
print("Dados tratados salvos na tabela 'dados_tratados'")


Dados tratados salvos na tabela 'dados_tratados'


In [9]:
# Estatísticas descritivas
print("Estatísticas descritivas dos dados tratados:")
print(df_clean[['temperatura', 'umidade', 'pressao', 'radiacao', 'vento_velocidade']].describe())


Estatísticas descritivas dos dados tratados:
       temperatura      umidade      pressao     radiacao  vento_velocidade
count  7161.000000  7161.000000  7161.000000  7161.000000       7161.000000
mean     23.854525    81.009217  1010.493088   524.422384          2.263874
std       2.595185    18.902229     4.022953   955.593412          1.570100
min      16.500000    13.000000   999.300000     0.000000          0.200000
25%      21.900000    70.670034  1007.900000    28.398288          1.200000
50%      23.736416    88.235260  1010.200000    67.400000          1.800000
75%      25.500000    94.104396  1012.800000   389.200000          2.800000
max      31.100000   100.000000  1024.100000  4185.700000         11.000000
